In [23]:
import pickle
from nltk import pos_tag
from nltk.tokenize import word_tokenize
import codecs
def create_feature_file(rname,wname,feat):
    f = codecs.open(rname, encoding = "latin")
    data = f.read()
    #data = data[:2]
    train_data =  data.split("\n\n")
    
    train = []
    label = []
    for i in range(len(train_data)):
        if(len(train_data[i])==0):
            continue
        lis = train_data[i].split("\n")
        seq = []
        lab = []
    
        #print(len(lis),lis,i)
        for word in lis:
            #print(word)
            seq.append(word.split()[0])
            lab.append(word.split()[1])
            #print("seq = ",lab)
        train.append(seq)
        label.append(lab)
    print("Seprated")
    
    #ADD POS Feature    
    if "1" in feat:
        POS = []
        for sen in train:
            tokens_pos = pos_tag(sen)
            lis = [x[1] for x in tokens_pos]
            POS.append(lis)
            
        pickle_out = open(rname[:-4]+"_POS.pickle","wb")
        pickle.dump(POS, pickle_out)
        pickle_out.close()
        
        print("POS")
    
    #ADD Synonym feature
    if "2" in feat:
        SYN = []
        for sen in train:
            SYN.append(get_syn(sen))
            
        pickle_out = open(rname[:-4]+"_SYN.pickle","wb")
        pickle.dump(SYN, pickle_out)
        pickle_out.close()
        
        print("SYN")
    
    #ADD Glove Word Embedding as feature
    if "3" in feat:
        GE = []
        for sen in train:
            GE.append(get_sen_vector(sen))
        
        pickle_out = open(rname[:-4]+"_GE.pickle","wb")
        pickle.dump(GE, pickle_out)
        pickle_out.close()
        
        print("GE")
    
    #ADD MH & MN categories Feature
    if "4" in feat:
        MH_MN = []
        for sen in train:
            MH_MN.append(get_mesh_sen_vector(sen))
            
        pickle_out = open(rname[:-4]+"_MH.pickle","wb")
        pickle.dump(MH_MN, pickle_out)
        pickle_out.close()
        
        print("MH")  
    
    if "5" in feat:
        Ortho = []
        for sen in train:
            Ortho.append(get_orthographic_vector(sen))
            
        pickle_out = open(rname[:-4]+"_Ortho.pickle","wb")
        pickle.dump(Ortho, pickle_out)
        pickle_out.close()
        
        print("Ortho")  
    
    if "6" in feat:
        Stem = []
        for sen in train:
            Stem.append(get_stem_vector(sen))
            
        pickle_out = open(rname[:-4]+"_Stem.pickle","wb")
        pickle.dump(Stem, pickle_out)
        pickle_out.close()
        
        print("Stem")  
    """
    train_feature = ""
    for i,sen in enumerate(train):
        st = ""
        for j,word in enumerate(sen):
            st = st + word + " "
            if "1" in feat:
                st = st + POS[i][j] + " "
            if "2" in feat:
                if SYN[i][j]!="UNKNOWN":
                    st = st + SYN[i][j] + " "
            if "3" in feat:
                st = st + GE[i][j]
                
            if "4" in feat:
                st = st + MH_MN[i][j] + " "
            
            st = st + label[i][j] + "\n"
        train_feature = train_feature + st + "\n"
        
    f = open(wname,"w")
    f.write(train_feature)
    f.close()
    """

In [7]:
from nltk.corpus import wordnet
def get_syn(sen):
    lis = []
    for word in sen:
        syns = wordnet.synsets(word)
        chk = 0
        for i in range(len(syns)):
            if word.lower() != syns[i].lemmas()[0].name():
                lis.append(syns[i].lemmas()[0].name())
                chk =1
                break
        if chk==0:        
            lis.append("UNKNOWN")
    return lis

In [8]:
import numpy as np
import os
embeddings_index = {}
f = open(os.path.join("glove.6B", 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [9]:
def get_vector(word):
    if word not in embeddings_index:
        return " "
    arr = embeddings_index[word]
    st=""
    for i in range(len(arr)):
        st = st + str(arr[i]) + " "
    return st

def get_sen_vector(sen):
    lis = []
    for word in sen:
        lis.append(get_vector(word.lower()))
    #print(len(lis))
    return lis

In [10]:
import pickle
pickle_in = open("mesh.pickle","rb")
mesh = pickle.load(pickle_in)
pickle_in.close()
pickle_in = open("MH.pickle","rb")
MH = pickle.load(pickle_in)
pickle_in.close()

In [11]:
def get_mesh_ontology(word):
    s = set()
    for i, x in enumerate(MH):
        if word.lower() in MH[x]:
            s.update(mesh[x]["mn"])
    vec = ' '.join([x for x in sorted(s)])
    return vec

def get_mesh_sen_vector(sen):
    lis = []
    for word in sen:
        #print(word)
        lis.append(get_mesh_ontology(word.lower()))
    return lis

In [19]:
def orthographic(st):
    feat = ""
    if all(x.isupper() for x in st):
        feat = feat + "ALLCAPS "
    if any(x.isalpha() for x in st) and any(x.isdigit() for x in st):
        feat = feat + "ALPNUM "
    if all(x.islower() for x in st):
        feat = feat + "LOW "
    if any(letter.islower() for letter in st) and any(letter.isupper() for letter in st):
        feat = feat + "MIXED "
    if st[0].isupper() and all(x.islower() for x in st[1:]):
        feat = feat + "TITLE "
    if st.count('-')==1:
        feat = feat + "DASH1 "
    if st.count('-')==2:
        feat = feat + "DASH2 "
    if st.count('/')>0:
        feat = feat + "SLASH "
    if st[0]=='[' and st[-1]==']':
        feat = feat + "BRACKS "
    
    return feat

def get_orthographic_vector(sen):
    lis = []
    for word in sen:
        lis.append(orthographic(word))
    return lis

In [22]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()
def get_stem_vector(sen):
    lis = []
    for word in sen:
        lis.append(ps.stem(word))
    return lis

In [24]:
feat = ["6"]
create_feature_file("training.txt","train_feature.txt",feat)
create_feature_file("dev.txt","dev_feature.txt",feat)
create_feature_file("test.txt","test_feature.txt",feat)

Seprated
Stem
Seprated
Stem
Seprated
Stem


In [25]:
f = codecs.open("test_feature.txt",encoding = "latin")
data = f.read().split("\n")

In [33]:
true_label = []
for i in range(len(data)):
    if(len(data[i]))>0:
        true_label.append(data[i][-1])

In [34]:
len(true_label)

6238

In [49]:
f = codecs.open("out.txt",encoding = "latin")
data = f.read().split("\n")
pred_label = []
for i in range(len(data)):
    if(len(data[i]))>0:
        pred_label.append(data[i][0])


In [50]:
len(pred_label)

6238

In [70]:
from sklearn.metrics import classification_report
print(classification_report(true_label,pred_label))

             precision    recall  f1-score   support

          D       0.81      0.62      0.70       489
          O       0.92      0.98      0.95      5357
          T       0.66      0.33      0.44       392

avg / total       0.90      0.91      0.90      6238



In [56]:
cnt/len(true_label)

0.9073420968259057

In [51]:
pred_label[0]

'O'

In [46]:
true_label[0]

'O'

In [47]:
pred_label[0]==true_label[0]

False

In [48]:
a = 'O'
b ='O'
a==b

True

In [82]:
f = codecs.open("dev_feature.txt",encoding = "latin")
data = f.read().split("\n")
true_label = []
for i in range(len(data)):
    if(len(data[i]))>0:
        true_label.append(data[i][-1])
f = codecs.open("dev_out.txt",encoding = "latin")
data = f.read().split("\n")
pred_label = []
for i in range(len(data)):
    if(len(data[i]))>0:
        pred_label.append(data[i][0])
    
print(classification_report(true_label,pred_label))

f = codecs.open("test_feature.txt",encoding = "latin")
data = f.read().split("\n")
true_label = []
for i in range(len(data)):
    if(len(data[i]))>0:
        true_label.append(data[i][-1])
f = codecs.open("test_out.txt",encoding = "latin")
data = f.read().split("\n")
pred_label = []
for i in range(len(data)):
    if(len(data[i]))>0:
        pred_label.append(data[i][0])
    
print(classification_report(true_label,pred_label))

             precision    recall  f1-score   support

          D       0.76      0.62      0.68       977
          O       0.93      0.97      0.95     11466
          T       0.71      0.37      0.49       819

avg / total       0.90      0.91      0.90     13262

             precision    recall  f1-score   support

          D       0.81      0.62      0.70       489
          O       0.92      0.98      0.95      5357
          T       0.66      0.33      0.44       392

avg / total       0.90      0.91      0.90      6238



In [73]:
f = codecs.open("test_feature.txt",encoding = "latin")
data = f.read().split("\n")
true_label = []
for i in range(len(data)):
    if(len(data[i]))>0:
        true_label.append(data[i][-1])
f = codecs.open("test_out.txt",encoding = "latin")
data = f.read().split("\n")
pred_label = []
for i in range(len(data)):
    if(len(data[i]))>0:
        pred_label.append(data[i][0])
    
print(classification_report(true_label,pred_label))

             precision    recall  f1-score   support

          D       0.77      0.12      0.21       489
          O       0.87      1.00      0.93      5357
          T       0.82      0.08      0.15       392

avg / total       0.86      0.87      0.82      6238



In [1]:
rname = "training.txt"

In [5]:
rname[:-4]+"_MH.pickle"

'training_MH.pickle'

In [14]:
pickle_in = open("dev_POS.pickle","rb")
d = pickle.load(pickle_in)
pickle_in.close()

In [15]:
d

[['PRP$', 'JJ', 'NN', 'VBZ', 'JJ', 'IN', 'VBG', 'NN'],
 ['VBN',
  'IN',
  'DT',
  'NN',
  '(',
  'WDT',
  'RB',
  'VBZ',
  'JJ',
  ')',
  'VBZ',
  'VBN',
  'IN',
  'DT',
  'JJ',
  'NN',
  'WRB',
  'VBP',
  'JJ',
  'NNS',
  'NN',
  '.',
  'RB',
  'PRP',
  'VBZ',
  'VBN',
  'IN',
  'DT',
  'NNS',
  'VBP',
  'DT',
  'NN',
  'IN',
  'JJ',
  'NN',
  'IN',
  'DT',
  'NN',
  'IN',
  'DT',
  'RB',
  'JJ',
  'NN',
  'IN',
  'NN',
  ',',
  'DT',
  'NN',
  'WDT',
  'VBZ',
  'VBN',
  'IN',
  'DT',
  'NN',
  'IN',
  'JJ',
  'JJ',
  'JJ',
  'NN',
  'IN',
  'NN',
  'IN',
  'DT',
  'JJ',
  'NN',
  'IN',
  'NN',
  'IN',
  'CD',
  'CC',
  'DT',
  'JJ',
  'NN'],
 ['JJ', 'CD', ':', 'NNS', 'CC', 'NN'],
 ['DT', 'NN', 'IN', '``', 'NN', "''"],
 ['NNS',
  ':',
  'DT',
  'DT',
  'NN',
  'NN',
  'CC',
  'NN',
  'NNS',
  'IN',
  'DT',
  'JJ',
  'NN',
  'VBP',
  'DT',
  'JJ',
  'NN',
  'IN',
  'NN',
  'IN',
  'DT',
  'NN',
  'NN',
  'IN',
  'NNS',
  'IN',
  'JJ',
  'JJ',
  'NNS',
  'IN',
  'NN'],
 ['NNS',
  ':',
 